In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as manimation
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
import warnings
import os
import sys
import datetime as dt
import cartopy.crs as ccrs
#from cartopy.feature import NaturalEarthFeature
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pandas as pd
import matplotlib as mpl

import matplotlib.ticker as mticker
#warnings.filterwarnings("ignore", category=RuntimeWarning) # ignore runtime warnings; here they arise from attempted operations on all-NaN arrays

/home/ords/crd/ccrn/reo000/mamba_files/envs/mpy12MHW/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
nl = dict(zip( ['CanCM4i', 'COLA-RSMAS-CCSM4', 'GEM-NEMO', 'GFDL-SPEAR', 'NASA-GEOSS2S', 'NCEP-CFSv2', 'CanESM5', 'GEM5.2-NEMO', 'GEM5-NEMO', 'CanCM4i-IC3',],
               [       11,                 11,         11,           11,              8,            9,        11,            11,          11,            11,])) 
modict={'Jacox':['CanCM4i', 'COLA-RSMAS-CCSM4', 'GEM-NEMO', 'GFDL-SPEAR', 
                        'NASA-GEOSS2S', 'NCEP-CFSv2'],
        'CanSIPSv2':['CanCM4i','GEM-NEMO'],
        'CanSIPSv21':['CanCM4i-IC3','GEM5-NEMO'],
        'CanSIPSv3':['CanESM5','GEM5.2-NEMO'],
        'GFDLNASA':[ 'GFDL-SPEAR','NASA-GEOSS2S'],}

In [3]:
basepath='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW'
clim_years = [1991, 2020]
years=clim_years
mhwdir = basepath+'/mhw/detrended';
#f_obs = basepath+f'/OISST/mhw_detrended_oisst-avhrr-v02r01.regridded1x1.monthly.{years[0]}_{years[-1]}.nc'
f_obs = basepath+f'/OISST/mhw_detrended_oisst-avhrr-v02r01.regridded1x1.monthly.{clim_years[0]}_{clim_years[-1]}.nc'
fobs=xr.open_dataset(f_obs)
mods=modict['CanSIPSv3']

ffor={};mhwfor={}
for il in range(0,9):
    flist=[]
    for modi in mods:
        if il<nl[modi]:
            # Load MHWs
            f_in = f'{mhwdir}/mhw_{modi}_l{il}_detrended_{clim_years[0]}_{clim_years[1]}.nc'
            flist.append(f_in)
    ffor[il]=xr.open_mfdataset(flist,chunks={'time':1,'X':-1,'Y':-1,'M':-1},concat_dim='M',combine='nested',data_vars='minimal',
                       coords='minimal',parallel=True,preprocess=lambda f: f.drop_vars(["sst_an_thr","mhw_prob"]))
    M=40
    mhwfor[il]=ffor[il].is_mhw.data[:(-1*il),...].sum(axis=1)/M if il>0 else ffor[il].is_mhw.data.sum(axis=1)/M
    #mhwobs=fobs.is_mhw.data[il:,...]
    #lm=np.sum(fobs.is_mhw.data,axis=0)==0

In [4]:
mhwobs=fobs.is_mhw.data
lm=np.sum(fobs.is_mhw.data,axis=0)==0

In [5]:
# ice mask
f_ice=basepath+f'/OISST/oisst-avhrr-v02r01.regridded1x1.monthly.{years[0]}_{years[-1]}.nc'
#with xr.open_dataset(f_ice) as fice:
fice=xr.open_dataset(f_ice)
icemask1=np.nanmax(fice.ice.data,axis=0)>0
Athresh=.50
Tthresh=.9
#Amax=np.zeros((20,)+np.shape(icemask1))
# for ind in range(0,20):
#     Amax[ind,...]=np.nanmax(fice.ice.data[12*ind:12*(ind+1),...],axis=0)
lost=np.where(fice.ice>Athresh,1,0).sum(axis=0)
#AmaxH=np.mean(np.where(Amax>0,1,0),axis=0)
imask=lost>(1-Tthresh)*np.shape(fice.ice)[0]

/tmp/reo000/3255475/ipykernel_3617178/3439421695.py:5: RuntimeWarning: All-NaN slice encountered
  icemask1=np.nanmax(fice.ice.data,axis=0)>0


In [6]:
fobs

<xarray.Dataset> Size: 194MB
Dimensions:     (X: 360, Y: 181, S: 360, Mon: 12)
Coordinates:
  * X           (X) float32 1kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * Y           (Y) float32 724B -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * S           (S) float32 1kB 372.0 373.0 374.0 375.0 ... 729.0 730.0 731.0
  * Mon         (Mon) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    lon         (X) float32 1kB ...
    lat         (Y) float32 724B ...
    time        (S) float32 1kB ...
    year        (S) int64 3kB ...
    month       (S) int64 3kB ...
    sst_an_thr  (Mon, Y, X) float64 6MB ...
    is_mhw      (S, Y, X) float64 188MB 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0

In [7]:
mhwobs.shape

(360, 181, 360)

In [8]:
cmap=plt.get_cmap('Reds')
cmap.set_bad('w',alpha=0)
i0=275;i1=360;j0=109;j1=162

In [9]:
dates=np.array([dt.datetime(years[0]+int((i)/12),(i)%12+1,15) for i in range(0,mhwobs.shape[0])])
dates[0],dates[-1]

(datetime.datetime(1991, 1, 15, 0, 0), datetime.datetime(2020, 12, 15, 0, 0))

In [10]:
ix = lambda yr,mn,lead: int(yr-1991)*12+int(mn-1)-lead

In [11]:
dates[ix(2011,12,0)]

datetime.datetime(2011, 12, 15, 0, 0)

In [12]:
np.shape(fobs.lon.values),np.shape(mhwobs)

((360,), (360, 181, 360))

In [13]:
fields=['year','month','A1_obs','A2_obs']
for lag in range(0,9):
    fields.append(f'A1_lag{lag}_weighted')
    fields.append(f'A2_lag{lag}_weighted')
    for th in range(1,41):
        fields.append(f'A1_lag{lag}_gt{th}')
        fields.append(f'A2_lag{lag}_gt{th}')

In [14]:
dfA=pd.DataFrame(columns=fields)

In [15]:
area=np.cos(np.pi/180*np.ones(np.shape(mhwobs[0,...]))*np.expand_dims(fobs.lat.values,axis=1))*111.3**2

In [16]:
def makebox(lons,lats,n=5):
    x=[]
    y=[]
    for ii in np.linspace(lons[0],lons[1],n):
        x.append(ii)
        y.append(lats[0])
    for ind,ii in enumerate(np.linspace(lats[0],lats[1],n)):
        if ind>0:
            x.append(lons[1])
            y.append(ii)
    for ind,ii in enumerate(np.linspace(lons[1],lons[0],n)):
        if ind>0:
            x.append(ii)
            y.append(lats[1])
    for ind,ii in enumerate(np.linspace(lats[1],lats[0],n)):
        if ind>0:
            x.append(lons[0])
            y.append(ii)
    return x,y
def addbox(axes,lons,lats,n=5,opts={}):
    x,y=makebox(lons,lats,n)
    axes.plot(x,y,transform=ccrs.PlateCarree(),**opts)

In [17]:
lonind = lambda lon,tol=.1:int(np.argwhere((np.abs(fobs.lon.values-lon)<tol)|\
                                       (np.abs(fobs.lon.values+360-lon)<tol)|\
                                       (np.abs(fobs.lon.values-360-lon)<tol))[0][0])
latind = lambda lat,tol=.1:int(np.argwhere(np.abs(fobs.lat.values-lat)<tol)[0][0])

In [18]:
lonind(-66),latind(51)

(294, 141)

In [19]:
box1=[-67,-43,42,50]
box2=[-64,-43,51,65]
area1=np.zeros(np.shape(area))
area2=np.zeros(np.shape(area))
area1[latind(box1[2]):latind(box1[3])+1,lonind(box1[0]):lonind(box1[1])+1]=area[latind(box1[2]):latind(box1[3])+1,lonind(box1[0]):lonind(box1[1])+1]
area2[latind(box2[2]):latind(box2[3])+1,lonind(box2[0]):lonind(box2[1])+1]=area[latind(box2[2]):latind(box2[3])+1,lonind(box2[0]):lonind(box2[1])+1]

In [20]:
for i in range(0,24):
    yy=2011+int((i+5)/12)
    mm=(i+5)%12+1
    print(yy,mm)

2011 6
2011 7
2011 8
2011 9
2011 10
2011 11
2011 12
2012 1
2012 2
2012 3
2012 4
2012 5
2012 6
2012 7
2012 8
2012 9
2012 10
2012 11
2012 12
2013 1
2013 2
2013 3
2013 4
2013 5


In [ ]:
for ind in range(0,24):
    yy=2011+int((ind+5)/12)
    mm=(ind+5)%12+1
    print(ind,yy,mm,dt.datetime.now())
    dfA.loc[ind,['year','month']]=[yy,mm]
    fig,ax=plt.subplots(2,5,figsize=(12,4.7),
                        subplot_kw=dict(projection=ccrs.Orthographic(central_longitude=-56, central_latitude=44.7, globe=None)),
                        gridspec_kw=dict(hspace=.25,wspace=.02))
    ax=ax.flatten()
    for iax in ax:
        iax.set_extent([-80, -20, 20, 70], ccrs.PlateCarree())
    ii=0
    # b1=0*np.ones(np.shape(mhwobs[0,...]))
    # b1[latind(box1[2]):latind(box1[3])+1,lonind(box1[0]):lonind(box1[1])+1]=.5
    # b1[latind(box2[2]):latind(box2[3])+1,lonind(box2[0]):lonind(box2[1])+1]=1
    # ax[ii].pcolormesh(fobs.lon.values,fobs.lat.values,b1,vmin=0,vmax=1,cmap=cmap,transform=ccrs.PlateCarree())
    m=ax[ii].pcolormesh(fobs.lon.values,fobs.lat.values,np.ma.masked_where(imask|lm,mhwobs[ix(yy,mm,0),...]),vmin=0,vmax=1,cmap=cmap,transform=ccrs.PlateCarree())
    ax[0].set_title('OISST')
    addbox(ax[0],[box1[0]-.5,box1[1]+.5],[box1[2]-.5,box1[3]+.5],opts=dict(color='k',linestyle='-',alpha=.2))
    addbox(ax[0],[box2[0]-.5,box2[1]+.5],[box2[2]-.5,box2[3]+.5],opts=dict(color='k',linestyle='-',alpha=.2))
    dfA.loc[ind,'A1_obs']=np.nansum(area1*np.ma.masked_where(imask|lm,mhwobs[ix(yy,mm,0),...]))
    dfA.loc[ind,'A2_obs']=np.nansum(area2*np.ma.masked_where(imask|lm,mhwobs[ix(yy,mm,0),...]))
    
    for il in range(0,9):
        #print(dt.datetime.now(),il)
        ax[il+1].pcolormesh(fobs.lon.values,fobs.lat.values,np.ma.masked_where(imask|lm,mhwfor[il][ix(yy,mm,il),...]),vmin=0,vmax=1,cmap=cmap,
                            transform=ccrs.PlateCarree())
        ax[il+1].set_title(f'CanSIPs Lead {il}')
        dfA.loc[ind,[f'A1_lag{il}_weighted']]=np.nansum(area1*np.ma.masked_where(imask|lm,mhwfor[il][ix(yy,mm,il),...]))
        dfA.loc[ind,[f'A2_lag{il}_weighted']]=np.nansum(area2*np.ma.masked_where(imask|lm,mhwfor[il][ix(yy,mm,il),...]))
        for th in range(1,41):
            dfA.loc[ind,[f'A1_lag{il}_gt{th}']]=np.nansum(area1*np.where(np.ma.masked_where(imask|lm,mhwfor[il][ix(yy,mm,il),...])>th/40-1e-5,1,0))
            dfA.loc[ind,[f'A2_lag{il}_gt{th}']]=np.nansum(area2*np.where(np.ma.masked_where(imask|lm,mhwfor[il][ix(yy,mm,il),...])>th/40-1e-5,1,0))
    for iax in ax:
        iax.add_feature(cfeature.LAND,zorder=1,color='lightgray')
        gl = iax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, 
                          linewidth=1, color='gray', alpha=0.5, linestyle=':')
        gl.xlocator = mticker.FixedLocator([-90, -70, -50,-30,-10,10])
        gl.ylocator = mticker.FixedLocator([20, 30, 40, 50, 60,70])
    fig.suptitle(f'MHW Probability: {yy}-{mm}')
    axcb=fig.add_axes(rect=[.91,.12,.02,.76])
    fig.colorbar(m,cax=axcb)
    fig.savefig(f'figs/NAtl_bylead_{yy}-{mm}.png',bbox_inches='tight')

0 2011 12 2025-03-11 18:00:51.959708


In [ ]:
dfA.to_csv('figs/dfA.csv')

In [ ]:
dfA

In [ ]:
dfA=pd.read_csv('figs/dfA.csv',index_col=0)
dfA

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,6))
ax.plot(dfA.A1_obs,'k-',label='obs')
ax.plot(dfA.A1_lag0_weighted,'r',alpha=1,label='prob weighted lag 0')
ax.plot(dfA.A1_lag1_weighted,'r',alpha=.7,label='prob weighted lag 1')
ax.plot(dfA.A1_lag2_weighted,'r',alpha=.6,label='prob weighted lag 2')
ax.plot(dfA.A1_lag3_weighted,'r',alpha=.5,label='prob weighted lag 3')
ax.plot(dfA.A1_lag4_weighted,'r',alpha=.4,label='prob weighted lag 4')
ax.plot(dfA.A1_lag5_weighted,'r',alpha=.3,label='prob weighted lag 5')
ax.plot(dfA.A1_lag6_weighted,'r',alpha=.2,label='prob weighted lag 1')
ax.set_title('Region 1 MHW Area (km^2)')
ax.legend()
fig.save_fig('figs/MHWArea_Region1_weighted.png',bbox_inches='tight')

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,6))
ax.plot(dfA.A2_obs,'k-',label='obs')
ax.plot(dfA.A2_lag0_weighted,'r',alpha=1,label='prob weighted lag 0')
ax.plot(dfA.A2_lag1_weighted,'r',alpha=.7,label='prob weighted lag 1')
ax.plot(dfA.A2_lag2_weighted,'r',alpha=.6,label='prob weighted lag 2')
ax.plot(dfA.A2_lag3_weighted,'r',alpha=.5,label='prob weighted lag 3')
ax.plot(dfA.A2_lag4_weighted,'r',alpha=.4,label='prob weighted lag 4')
ax.plot(dfA.A2_lag5_weighted,'r',alpha=.3,label='prob weighted lag 5')
ax.plot(dfA.A2_lag6_weighted,'r',alpha=.2,label='prob weighted lag 1')
ax.set_title('Region 2 MHW Area (km^2)')
ax.legend()
fig.save_fig('figs/MHWArea_Region2_weighted.png',bbox_inches='tight')

In [ ]:
cmap = mpl.colormaps['jet']
colors = cmap(np.linspace(0, 1, 40))

In [ ]:
for lag in range(0,6):
    fig,ax=plt.subplots(1,1,figsize=(12,6))
    ax.plot(dfA.A1_obs,'k-',label='obs')
    for ii in range(1,41):
        ax.plot(dfA[f'A1_lag{lag}_gt{ii}'],color=colors[ii-1],alpha=.5,label=f'P>={ii}/40')
    ax.set_title(f'Region 1 MHW Area (km^2), Lag {lag}')
    ax.legend(ncols=2,bbox_to_anchor=[1.01,1])
    fig.savefig(f'figs/MHWAreaByPThresh_Region1_lag{lag}.png',bbox_inches='tight')

In [ ]:
for lag in range(0,6):
    fig,ax=plt.subplots(1,1,figsize=(12,6))
    ax.plot(dfA.A1_obs,'k-',label='obs')
    for ii in range(1,41):
        ax.plot(dfA[f'A2_lag{lag}_gt{ii}'],color=colors[ii-1],alpha=.5,label=f'P>={ii}/40')
    ax.set_title(f'Region 2 MHW Area (km^2), Lag {lag}')
    ax.legend(ncols=2,bbox_to_anchor=[1.01,1])
    fig.savefig(f'figs/MHWAreaByPThresh_Region2_lag{lag}.png',bbox_inches='tight')